# Adamnite Code Documentation

This Notebook explains the code and gives some examples on usage.
This should give new Developers of this Repo a good overview.
This entire software is written in an async. This avoids
creating threads and running into race conditions. Events and tasks
are automatically scheduled on the event loop. They yield from
time to time to let other tasks run also. It looks like
it is multithreading, but in reality it jumps back and forth depending
on which events need the attention of the eventloop.

# Entry Point


Node is the main Class in this application. With OOP composition it
contains all the submodules that this cryptocurrency needs.
1. block_chain
2. peers
3. wallet

In [ ]:
from adamnite.node import Node
import asyncio

node = Node(port=6101)
loop = asyncio.get_event_loop()
loop.create_task(node.start_serving())
loop.run_forever()

It controls all the communication outgoing and delegates tasks to the blockchain
when requested. Sockets with peers usually stay open and they communicate in
both ways.

# Block Chain

The blockchain manages as it implies the blockchain.
It appends new block's with the append method and validates
it or rejects it, if it is found invalid.

In [ ]:
from adamnite.account import PrivateAccount
from adamnite.blockchain import BlockChain
from adamnite.genesis import GENESIS_BLOCK

block = GENESIS_BLOCK
private_Account = PrivateAccount()
block_chain = BlockChain(proposer=private_Account)
block_chain.append(block)

The proposer is the private account that is managing this blockchain.
It proposes new blocks, if the consensus algo allows for it.

Adamnite is an account based blockchain. To keep track of all the balances
the blockchain module keeps a dictionary with all the wallets and their
balances. It tracks double spending with the nonce. The amount an account has
made transactions, to avoid relay attacks.

In [ ]:
block_chain.accounts
block_chain.nonce

# Accounts

There are 2 types of accounts. Private and Public Accounts.
The private Accounts contains the private key and are stored in memory
or in the os keyring for safekeeping. Private Accounts are needed to sign
transactions or propose new blocks. The private key is needed to sign them
off. Others can validate with the public key, that you in fact know the private
key.

Creates a new Account with new private and public key with os.urandom

In [ ]:
private_account = PrivateAccount()

Public Accounts are derived from Private or from addresses,

In [ ]:
from adamnite.account import PublicAccount

public_account = PublicAccount(
    address=b'uD4YLvrYBNrBMtU7pysMMUkoXMNo4f4xstz1PzuBzhNK'
)
public_account2 = private_account.public_account()

# Wallet

Wallets is a static class that holds all the accounts for the node
the software is running. It stores them in the os keyring. It stays persistent.

In [ ]:
from adamnite.account import Wallet

private_account = Wallet.accounts[0]

# Serialisation

Serialisation is one of the most important aspects of this cryptocurrency.
It serialises python class objects into a series of bytes. Other
participants in the network can convert them back to python objects and use
them. It closely follows ETH serialisation but expands it into nested classes.

In [ ]:
from adamnite.serialization import serialize, deserialize
import time


class CustomClass:
    class NestedClass:
        list = [1, 2, 3, 4]
    string = "string"
    number = 123
    list = ["list0", "list1", "list2"]
    tuple = ("string", "string", "string2")
    timestamp = int(time.time())
    bytes = b"12345"
    string2 = "string2"
    nested = [NestedClass(), NestedClass()]

custom_class_bytes = serialize(CustomClass)
restored_custom_class, read = deserialize(
    custom_class_bytes,
    to=CustomClass
)

# Messages

Messages are split into Requests and Responses.
Requests indicate to a node that a Resource is requested and that the peer
should send it to him.

## Request
Blocks with the height 2, 3, 4, 5

In [ ]:
from adamnite.message import Request, BLOCKS

message = Request(
    resource=BLOCKS,
    query=(2, 3, 4, 5)
)

## Respond

In [ ]:
from adamnite.message import Response

blocks = [GENESIS_BLOCK, GENESIS_BLOCK, GENESIS_BLOCK]
respond = Response(
    payload=blocks
)

Messages are serialized and send on the wire. The peer will read them
and respond with the requested resources.